In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from rough_bergomi.models import RoughBergomiModel, GBMModel, HestonModel
from rough_bergomi.utils.utils import bs_delta, bsinv, bs  # Black-Scholes inversion for IV calculation
import yfinance as yf
import seaborn as sns

from rough_bergomi.hedging.deep_hedger import DeepHedgerTF

In [4]:
model = RoughBergomiModel()
results = []

hidden_size = 64
n_layers = 1
optimizer = 'adam'
loss_types = ['mshe', 'cvar', 'entropic']

for loss in loss_types:
    print(f"Running Deep Hedger with loss: {loss.upper()}")
    
    hedger = DeepHedgerTF(
        model=model,
        M=1000,
        N=100,
        T=30
    )
    
    hedger.prepare_data()
    gamma = 1.0 if loss == 'entropic' else None
    
    hedger.build_and_compile_model(
        lr=0.001,
        loss_type=loss,
        gru_layers=n_layers,
        hidden_size=hidden_size,
        optimizer=optimizer,
        gamma=gamma
    )
    
    hedger.train_model(epochs=30, batch_size=256)

    hedging_error = hedger.evaluate_model()
    
    std_error = np.std(hedging_error)
    mean_error = np.mean(hedging_error)
    cvar_1_error = np.mean(np.sort(hedging_error)[:int(0.01 * len(hedging_error))])

    results.append((loss, mean_error, std_error, cvar_1_error))

for loss, mean_e, std_e, cvar_1 in results:
    print(f"{loss.upper()} | Mean: {mean_e:.4f}, Std: {std_e:.4f}, CVaR@1%: {cvar_1:.4f}")


Running Deep Hedger with loss: MSHE
Mean hedging error: -1.2698018880318622
Std of hedging error: 13.955262338069
Running Deep Hedger with loss: CVAR
Mean hedging error: 1.0254564201164118
Std of hedging error: 55.168720610150196
Running Deep Hedger with loss: ENTROPIC
Mean hedging error: -0.03513367411378681
Std of hedging error: 45.53737725971397
MSHE | Mean: -1.2698, Std: 13.9553, CVaR@1%: -64.1638
CVAR | Mean: 1.0255, Std: 55.1687, CVaR@1%: -360.5361
ENTROPIC | Mean: -0.0351, Std: 45.5374, CVaR@1%: -131.5437
